In [ ]:
import os
import sys
os.chdir('/home/peitian_zhang/Codes/NR')
sys.path.append('/home/peitian_zhang/Codes/NR')

import pytz
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
from torchtext.vocab import FastText
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from utils.MIND import MIND_iter,MIND_map
from utils.utils import getLoss,getLabel,constructBasicDict,run_eval,run_train
from models.NPA import NPAModel

## setting up the *NPA* model

### define paths and hyperparameters, load data

all these hyper parameters are fixed according to the paper [\[23\] Npa Neural news recommendation with personalized attention](https://dl.acm.org/doi/abs/10.1145/3292500.3330665)

- *mode*: data to read (*demo*/*small*/*large*)

- *batch_size*: size of each minibatch

- *title_size*: max word capacity of title

- *his_size*: max record capacity of click history

- *npratio*: number of negtive sampling

- *dropout_p*: probability of dropout layer

- *filter_num*: number of kernels in 1D CNN, which is also embedding dimension of news/user

- *embedding_dim*: word embedding dimension

- *user_dim*: user id embedding dimension

- *preference_dim*: user preference embedding dimension

In [ ]:
hparams = {
    'mode':'small',
    'epochs':10,
    'batch_size':100,
    'title_size':30,
    'his_size':50,   
    'npratio':4,
    'dropout_p':0.2,
    'filter_num':400,
    'embedding_dim':300,
    'user_dim':50,
    'preference_dim':200,
    'metrics':'group_auc,ndcg@4,mean_mrr',
    'gpu':'cuda:0',
    'attrs': ['title']
}

news_file_train = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_train/news.tsv'
news_file_test = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_dev/news.tsv'

behavior_file_train = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_train/behaviors.tsv'
behavior_file_test = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_dev/behaviors.tsv'

save_path = '/home/peitian_zhang/Codes/NR/models/model_param/NPA_'+ hparams['mode'] +'.model'

if not os.path.exists('data/dictionaries/vocab_{}_{}_{}.pkl'.format(hparams['mode'],'train','_'.join(hparams['attrs']))):
    constructBasicDict(news_file_train,behavior_file_train,hparams['mode'],'train',hparams['attrs'])

if not os.path.exists('data/dictionaries/vocab_{}_{}_{}.pkl'.format(hparams['mode'],'test','_'.join(hparams['attrs']))):
    constructBasicDict(news_file_test,behavior_file_test,hparams['mode'],'test',hparams['attrs'])

device = torch.device(hparams['gpu']) if torch.cuda.is_available() else torch.device("cpu")

dataset_train = MIND_map(hparams=hparams,mode='train',news_file=news_file_train,behaviors_file=behavior_file_train)

dataset_test = MIND_iter(hparams=hparams,mode='test',news_file=news_file_test,behaviors_file=behavior_file_test)

vocab_train = dataset_train.vocab
embedding = FastText('simple',cache='.vector_cache')
vocab_train.load_vectors(embedding)

vocab_test = dataset_test.vocab
vocab_test.load_vectors(embedding)

loader_train = DataLoader(dataset_train,batch_size=hparams['batch_size'],shuffle=True,pin_memory=True,num_workers=3,drop_last=True)
loader_test = DataLoader(dataset_test,batch_size=hparams['batch_size'],pin_memory=True,num_workers=0,drop_last=True)

writer = SummaryWriter('data/tb/npa/' + datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
npaModel = NPAModel(vocab=vocab_train,hparams=hparams).to(device)
npaModel.train()

In [ ]:
if npaModel.training:
    print("training...")
    loss_func = getLoss(npaModel)
    optimizer = optim.Adam(npaModel.parameters(),lr=0.001)
    npaModel = run_train(npaModel,loader_train,optimizer,loss_func,writer, epochs=hparams['epochs'], interval=10)

In [ ]:
print("evaluating...")
npaModel.eval()
npaModel.vocab = vocab_test
npaModel.npratio = -1

run_eval(npaModel,loader_test)